In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'  # Set font for LaTeX rendering
latex_image_dir = "latex_eqn_images"  # Directory to store generated images of LaTeX equations
os.makedirs(latex_image_dir, exist_ok=True)  # Ensure directory exists

# Load data from CSV file and select the last 5 rows
data = pd.read_csv("QUE4.csv")
# df = data.tail(5)

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=50):
    # Create a figure for rendering the LaTeX string
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    # Save the figure as an image with high DPI for clarity
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up resources

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    # Split the text into parts, separating LaTeX equations and regular text
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        # If word is a LaTeX equation, add it as a separate line
        if re.match(r'\$.*?\$', word):
            if current_line:  # Append the current line if it contains text
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add the LaTeX equation as its own line
        else:  # If word is regular text, break it down into sub-words
            for sub_word in word.split():
                # Check if adding this word to the current line fits within max width
                if stringWidth(current_line + " " + sub_word, "Helvetica", 10) < max_width:
                    current_line += " " + sub_word
                else:
                    # If it doesn't fit, add the current line and start a new line
                    lines.append(current_line.strip())
                    current_line = sub_word  # Start new line with current word
    if current_line:  # Add any remaining text in current_line to lines
        lines.append(current_line.strip())
    
    return lines

# Function to generate a PDF with LaTeX-rendered images and wrapped text
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))  # Create PDF canvas
    width, height = landscape(letter)  # Get page dimensions

    # Set layout margins and spacing parameters
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = 575
    min_y_position = 50  # Minimum Y position to avoid text running off the page

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        y_position = top_margin  # Reset Y position for each row

        # Loop through each column and value in the row
        for col, value in row.items():
            text = f"{col}: {value}"  # Create text string for column and value
            lines = wrap_text(text, max_line_width)  # Wrap text to fit within max width
            x_position = left_margin  # Reset X position for each row

            # Loop through each wrapped line
            for line in lines:
                # Ensure y_position stays within the printable area
                if y_position <= min_y_position:
                    c.showPage()  # Start a new page if current Y position is too low
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # If the line contains a LaTeX equation
                    # Render LaTeX to image if not already done
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70  # Define image width
                    if x_position + image_width <= max_line_width:
                        # Draw the image on the current line if it fits
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        # Move to the next line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    # For regular text, measure the line width and position it accordingly
                    line_width = stringWidth(line, "Helvetica", 10)
                    if x_position + line_width <= max_line_width:
                        # Draw the text on the same line if it fits
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        # Move to the next line if text doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Add extra space after each row
        c.showPage()  # Move to a new page after each row

    c.save()  # Save and close the PDF file

# Run the PDF generation function
create_pdf_with_latex(data)


In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re
from PIL import Image

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images_1"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears2.csv")
data = df.iloc[:, :-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, base_img_width=100):
    img_width = base_img_width + len(latex_string) * 2
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=14, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_font_lin_eqn_test.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$.*?\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path)

                    with Image.open(latex_image_path) as img:
                        image_width, image_height = img.size
                        base_scale = 0.05
                        scaling_factor = base_scale + len(line) * 0.005
                        scaled_width = image_width * scaling_factor
                        scaled_height = image_height * scaling_factor

                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position , y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += image_width + 45
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


In [17]:
df = pd.read_csv("linears.csv")
df

,Question,Option A,Option B,Option C,Option D,Correct Option
0,Solve x: $2x + 3 = \frac{x - 1}{2}\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""A"""
1,Solve x: $8x - 6 = 3(2x + 1)\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""B"""
2,Solve x: $\frac{3x + 2}{4} = 2\$,$\frac{-7}{3},"$\frac{9}{2}$""",2,6,"""C"""
3,Solve x: $\frac{5x}{3} = x + 4\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""D"""
4,Solve x: $7x + 5 = \frac{4x + 1}{2}\$,$\frac{-9}{10}$,-5,"$\frac{-20}{7}$""",$\frac{13}{3}$,"""A"""
5,Solve x: $4(x - 2) = 2(3x + 1)\$,$\frac{-}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""B"""
6,Solve x: $\frac{2x - 5}{3} - \frac{3x + 2}{4} ...,$\frac{-9}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""C"""
7,Solve x: $\frac{x + 2}{5} + \frac{x - 3}{2} = ...,$\frac{-9}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""D"""


In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears.csv")
data = df.iloc[:,:-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=165):
    # latex_string = wrap_as_latex(latex_string)
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=12, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap in latex syntax
def wrap_as_latex(text):
    return f"${text}$"

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'\$.*?\$', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_fontttt.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 50
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$.*?\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        # wrapped_latex = wrap_as_latex(line)
                        render_latex_to_image(line, latex_image_path, img_width=165)

                    image_width = 165
                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=50)
                        x_position += image_width + 45
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=50)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


In [19]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from PIL import Image
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stixsans'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("QUE4.csv")
# data = df.iloc[:, :-1]


# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=150):
    # img_width = len(latex_string) * 5
    fig = plt.figure(figsize=(img_width / 100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=8, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)  # Split by LaTeX segments within $...$
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):  # Detect LaTeX segments
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add LaTeX segment as a separate line
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text and LaTeX equations
def create_pdf_with_latex(df, filename="output_dynamic_latex_equations_mod1.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 50
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # Process LaTeX segments
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=150)

                    image_width =150
                    
                    # Draw image with dynamic scaling
                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=35)
                        x_position += image_width + 10
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=35)
                        x_position += image_width + 10
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16)  # Reset font for new page

    c.save()

# Run the function to create PDF
create_pdf_with_latex(df)


In [28]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from PIL import Image
import re

# Configure matplotlib to not use LaTeX (more reliable)
matplotlib.use('Agg')  # Use Agg backend
matplotlib.rcParams['mathtext.fontset'] = 'cm'  # Use Computer Modern font
matplotlib.rcParams['axes.unicode_minus'] = False  # Fix minus sign

# Create output directories
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Try to register font, with fallback
try:
    pdfmetrics.registerFont(TTFont("Lato", "fonts/Lato-Regular.ttf"))
    default_font = "Lato"
except:
    default_font = "Helvetica"  # Built-in fallback font
    print("Using Helvetica as fallback font")

def render_latex_to_image(latex_string, filename, img_width=150):
    """Render LaTeX expression to image using matplotlib's math mode."""
    try:
        # Clean the latex string
        latex_clean = latex_string.strip('$')
        
        # Create figure with white background
        fig = plt.figure(figsize=(img_width / 100, 0.5), facecolor='white')
        ax = fig.add_axes([0, 0, 1, 1])
        ax.axis('off')
        
        # Render the equation
        ax.text(0.5, 0.5, f'${latex_clean}$',
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=12)
        
        # Save with white background
        fig.savefig(filename, 
                   dpi=300,
                   bbox_inches='tight',
                   pad_inches=0.1,
                   facecolor='white',
                   edgecolor='none')
        plt.close(fig)
        return True
    except Exception as e:
        print(f"Error rendering latex: {e}")
        print(f"Problematic latex string: {latex_string}")
        return False

def wrap_text(text, max_width, font_name=default_font, font_size=16):
    """Wrap text while preserving LaTeX expressions."""
    segments = re.split(r'(\$[^$]+\$)', text)  # Split by LaTeX segments
    lines = []
    current_line = ""
    
    for segment in segments:
        if segment.strip():
            if segment.startswith('$') and segment.endswith('$'):
                # Handle LaTeX segment
                if current_line:
                    lines.append(current_line.strip())
                lines.append(segment)
                current_line = ""
            else:
                # Handle regular text
                words = segment.split()
                for word in words:
                    test_line = current_line + (" " if current_line else "") + word
                    if stringWidth(test_line, font_name, font_size) < max_width:
                        current_line = test_line
                    else:
                        if current_line:
                            lines.append(current_line.strip())
                        current_line = word
    
    if current_line:
        lines.append(current_line.strip())
    
    return lines

def create_pdf_with_latex(df, filename="output_with_latex.pdf"):
    """Create PDF with both text and LaTeX equations."""
    # Initialize PDF
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont(default_font, 16)
    
    # Set margins and spacing
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30  # Reduced from 50
    max_line_width = width - 2 * left_margin
    min_y_position = 50
    
    latex_count = 0  # Counter for unique latex images
    
    for index, row in df.iterrows():
        y_position = top_margin
        
        # Process each column in the row
        for col, value in row.items():
            if pd.isna(value):  # Skip NaN values
                continue
            
            # Format the text
            text = f"{col}: {str(value)}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin
            
            for line in lines:
                # Check for page break
                if y_position <= min_y_position:
                    c.showPage()
                    c.setFont(default_font, 16)
                    y_position = top_margin
                
                # Process LaTeX expressions
                if '$' in line:
                    latex_expressions = re.findall(r'(\$.*?\$)', line)
                    for latex in latex_expressions:
                        latex_count += 1
                        image_name = f"latex_{latex_count}.png"
                        latex_image_path = os.path.join(latex_image_dir, image_name)
                        
                        # Render LaTeX
                        success = render_latex_to_image(f"${latex}$", latex_image_path)
                        
                        if success and os.path.exists(latex_image_path):
                            # Check if need to move to next line
                            if x_position + 150 > width - left_margin:
                                y_position -= line_spacing
                                x_position = left_margin
                            
                            # Draw the image
                            try:
                                c.drawImage(latex_image_path, x_position, y_position - 15,
                                          width=150, height=30, preserveAspectRatio=True)
                                x_position += 160  # Image width + spacing
                            except Exception as e:
                                print(f"Error drawing image: {e}")
                        else:
                            # Fallback: draw the LaTeX as text
                            c.drawString(x_position, y_position, latex)
                            x_position += stringWidth(latex, default_font, 16) + 10
                else:
                    # Draw regular text
                    if x_position + stringWidth(line, default_font, 16) > width - left_margin:
                        y_position -= line_spacing
                        x_position = left_margin
                    
                    c.drawString(x_position, y_position, line)
                    x_position += stringWidth(line, default_font, 16) + 10
                
            y_position -= line_spacing
        
        # New page for each row
        c.showPage()
        c.setFont(default_font, 16)
    
    # Save the PDF
    try:
        c.save()
        print(f"PDF saved successfully: {filename}")
    except Exception as e:
        print(f"Error saving PDF: {e}")

# Main execution with error handling
if __name__ == "__main__":
    try:
        # Read CSV file
        df = pd.read_csv("QUE4.csv")
        print(f"Successfully read CSV with {len(df)} rows")
        
        # Create PDF
        create_pdf_with_latex(df)
        
        # Clean up temporary files
        for file in os.listdir(latex_image_dir):
            if file.endswith(".png"):
                try:
                    os.remove(os.path.join(latex_image_dir, file))
                except:
                    pass
        
    except Exception as e:
        print(f"Error in main execution: {e}")

Successfully read CSV with 14 rows
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$E=mc^2$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$a^2 + b^2 = c^2$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$e^x$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$E=mc^2$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$E=mc^2$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$E=mc^2$$
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: $$E=mc^2$$
Error rendering latex: Failed to process string with tex because latex

RuntimeError: Failed to process string with tex because latex could not be found

In [23]:
create_pdf_with_latex(df)

In [27]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from PIL import Image
import re

# Configure matplotlib
matplotlib.use('Agg')
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['axes.unicode_minus'] = False

# Create output directories
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Font setup with fallback
try:
    pdfmetrics.registerFont(TTFont("Lato", "fonts/Lato-Regular.ttf"))
    default_font = "Lato"
except:
    default_font = "Helvetica"
    print("Using Helvetica as fallback font")

def extract_latex(text):
    """Extract LaTeX expressions from text."""
    pattern = r'\$(.*?)\$'
    matches = re.finditer(pattern, text)
    equations = []
    positions = []
    
    for match in matches:
        equations.append(match.group(1))  # Get the equation without $ symbols
        positions.append((match.start(), match.end()))
    
    return equations, positions

def render_latex_to_image(latex_string, filename, img_width=150):
    """Render LaTeX expression to image."""
    try:
        fig = plt.figure(figsize=(img_width/100, 0.5), facecolor='white')
        ax = fig.add_axes([0, 0, 1, 1])
        ax.axis('off')
        
        # Remove $ symbols if they're still present
        latex_clean = latex_string.strip('$')
        
        # Handle special characters and commands
        latex_clean = latex_clean.replace('\\vec', '\\vec ')  # Add space after \vec
        
        ax.text(0.5, 0.5, f'${latex_clean}$',
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=12)
        
        fig.savefig(filename, 
                   dpi=300,
                   bbox_inches='tight',
                   pad_inches=0.1,
                   facecolor='white',
                   edgecolor='none')
        plt.close(fig)
        return True
    except Exception as e:
        print(f"Error rendering latex: {e}")
        print(f"Problematic latex string: {latex_string}")
        return False

def process_text_with_equations(text, index, latex_count):
    """Process text containing LaTeX equations."""
    equations, positions = extract_latex(text)
    rendered_images = []
    
    # Render each equation
    for i, eq in enumerate(equations):
        latex_count += 1
        image_name = f"latex_{index}_{latex_count}.png"
        image_path = os.path.join(latex_image_dir, image_name)
        
        if render_latex_to_image(eq, image_path):
            rendered_images.append((image_path, positions[i]))
    
    return rendered_images, latex_count

def create_pdf_with_latex(df, filename="output_with_latex111.pdf"):
    """Create PDF with text and LaTeX equations."""
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont(default_font, 16)
    
    # Layout settings
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50
    latex_count = 0
    
    for index, row in df.iterrows():
        y_position = top_margin
        
        for col, value in row.items():
            if pd.isna(value):
                continue
                
            text = f"{col}: {str(value)}"
            x_position = left_margin
            
            # Check if text contains LaTeX equations
            if '$' in text:
                rendered_images, latex_count = process_text_with_equations(text, index, latex_count)
                
                # Split text by LaTeX positions
                last_pos = 0
                for image_path, (start, end) in rendered_images:
                    # Draw text before equation
                    prefix_text = text[last_pos:start].strip()
                    if prefix_text:
                        c.drawString(x_position, y_position, prefix_text)
                        x_position += stringWidth(prefix_text, default_font, 16) + 10
                    
                    # Draw equation image
                    if x_position + 150 > width - left_margin:
                        y_position -= line_spacing
                        x_position = left_margin
                    
                    try:
                        c.drawImage(image_path, x_position, y_position - 15,
                                  width=150, height=30, preserveAspectRatio=True)
                        x_position += 160
                    except Exception as e:
                        print(f"Error drawing image: {e}")
                    
                    last_pos = end
                
                # Draw remaining text
                suffix_text = text[last_pos:].strip()
                if suffix_text:
                    if x_position + stringWidth(suffix_text, default_font, 16) > width - left_margin:
                        y_position -= line_spacing
                        x_position = left_margin
                    c.drawString(x_position, y_position, suffix_text)
            
            else:
                # Draw regular text
                c.drawString(x_position, y_position, text)
            
            y_position -= line_spacing
            
            if y_position <= min_y_position:
                c.showPage()
                c.setFont(default_font, 16)
                y_position = top_margin
        
        c.showPage()
        c.setFont(default_font, 16)
    
    try:
        c.save()
        print(f"PDF saved successfully: {filename}")
        
        # Cleanup
        for file in os.listdir(latex_image_dir):
            if file.endswith(".png"):
                try:
                    os.remove(os.path.join(latex_image_dir, file))
                except:
                    pass
                    
    except Exception as e:
        print(f"Error saving PDF: {e}")

# Main execution
if __name__ == "__main__":
    try:
        # Read CSV file
        df = pd.read_csv("QUE4.csv")
        print(f"Successfully read CSV with {len(df)} rows")
        
        # Create PDF
        create_pdf_with_latex(df)
        
    except Exception as e:
        print(f"Error in main execution: {e}")

Successfully read CSV with 14 rows
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: E=mc^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: a^2 + b^2 = c^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: e^x
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: E=mc^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: E=mc^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: E=mc^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problematic latex string: E=mc^2
Error rendering latex: Failed to process string with tex because latex could not be found
Problema

RuntimeError: Failed to process string with tex because latex could not be found

## Each EQN Extracted

Need Layoout modificaiton, justifying paragraph etc.

In [35]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from PIL import Image
import re

# Configure matplotlib to use built-in math renderer
matplotlib.use('Agg')
matplotlib.rcParams['mathtext.fontset'] = 'dejavusans'  # Use built-in math fonts
matplotlib.rcParams['axes.unicode_minus'] = False

# Create output directories
latex_image_dir = "math_eqn_images"
os.makedirs(latex_image_dir, exist_ok=True)

# Font setup with fallback
try:
    pdfmetrics.registerFont(TTFont("Lato", "fonts/Lato-Regular.ttf"))
    default_font = "Lato"
except:
    default_font = "Helvetica"
    print("Using Helvetica as fallback font")

def extract_math(text):
    """Extract math expressions from text."""
    pattern = r'\$(.*?)\$'
    matches = re.finditer(pattern, text)
    equations = []
    positions = []
    
    for match in matches:
        equations.append(match.group(1))  # Get the equation without $ symbols
        positions.append((match.start(), match.end()))
    
    return equations, positions

# def render_math_to_image(math_string, filename, img_width=150):
#     """Render math expression to image using matplotlib's mathtext."""
#     try:
#         # Clean up the math string
#         math_clean = math_string.strip('$')
        
#         # Create figure with white background
#         fig = plt.figure(figsize=(img_width/100, 0.5), facecolor='white')
#         ax = fig.add_axes([0, 0, 1, 1])
#         ax.axis('off')
        
#         # Render the math expression
#         ax.text(0.5, 0.5, f'${math_clean}$',
#                 horizontalalignment='center',
#                 verticalalignment='center',
#                 fontsize=12)
        
#         # Save with high DPI for better quality
#         fig.savefig(filename, 
#                    dpi=300,
#                    bbox_inches='tight',
#                    pad_inches=0.1,
#                    facecolor='white',
#                    transparent=False)
#         plt.close(fig)
#         return True
#     except Exception as e:
#         print(f"Error rendering equation: {e}")
#         print(f"Problematic equation: {math_string}")
#         # Fall back to rendering as plain text if math rendering fails
#         fig = plt.figure(figsize=(img_width/100, 0.5), facecolor='white')
#         ax = fig.add_axes([0, 0, 1, 1])
#         ax.axis('off')
#         ax.text(0.5, 0.5, math_string,
#                 horizontalalignment='center',
#                 verticalalignment='center',
#                 fontsize=12)
#         fig.savefig(filename, 
#                    dpi=300,
#                    bbox_inches='tight',
#                    pad_inches=0.1,
#                    facecolor='white')
#         plt.close(fig)
#         return True



def render_math_to_image(math_string, filename, img_width=150):
    """Render math expression to image using matplotlib's mathtext."""
    try:
        # Clean up the math string
        math_clean = math_string.strip('$')
 # Create figure with white background
        fig = plt.figure(figsize=(img_width/100, 0.5), facecolor='white')
        ax = fig.add_axes([0, 0, 1, 1])
        ax.axis('off')
    
    # Render the math expression using mathtext
        ax.text(0.5, 0.5, f'${math_clean}$',
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=12, usetex=False)
    
    # Save with high DPI for better quality
        fig.savefig(filename, 
               dpi=300,
               bbox_inches='tight',
               pad_inches=0.1,
               facecolor='white',
               transparent=False)
        plt.close(fig)
        return True
    except Exception as e:
        print(f"Error rendering equation: {e}")
        print(f"Problematic equation: {math_string}")
        # Fall back to rendering as plain text if math rendering fails
        fig = plt.figure(figsize=(img_width/100, 0.5), facecolor='white')
        ax = fig.add_axes([0, 0, 1, 1])
        ax.axis('off')
        ax.text(0.5, 0.5, math_string,
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=12)
        fig.savefig(filename, 
               dpi=300,
               bbox_inches='tight',
               pad_inches=0.1,
               facecolor='white')
        plt.close(fig)
        return True




def process_text_with_equations(text, index, math_count):
    """Process text containing math equations."""
    equations, positions = extract_math(text)
    rendered_images = []
    
    for i, eq in enumerate(equations):
        math_count += 1
        image_name = f"math_{index}_{math_count}.png"
        image_path = os.path.join(latex_image_dir, image_name)
        
        if render_math_to_image(eq, image_path):
            rendered_images.append((image_path, positions[i]))
    
    return rendered_images, math_count

def create_pdf_with_math(df, filename="output_with_math.pdf"):
    """Create PDF with text and math equations."""
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont(default_font, 16)
    
    # Layout settings
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50
    math_count = 0
    
    for index, row in df.iterrows():
        y_position = top_margin
        
        for col, value in row.items():
            if pd.isna(value):
                continue
                
            text = f"{col}: {str(value)}"
            x_position = left_margin
            
            # Check if text contains math equations
            if '$' in text:
                rendered_images, math_count = process_text_with_equations(text, index, math_count)
                
                # Split text by math positions
                last_pos = 0
                for image_path, (start, end) in rendered_images:
                    # Draw text before equation
                    prefix_text = text[last_pos:start].strip()
                    if prefix_text:
                        c.drawString(x_position, y_position, prefix_text)
                        x_position += stringWidth(prefix_text, default_font, 16) + 10
                    
                    # Draw equation image
                    if x_position + 150 > width - left_margin:
                        y_position -= line_spacing
                        x_position = left_margin
                    
                    try:
                        img = Image.open(image_path)
                        aspect = img.height / img.width
                        img_width = 150
                        img_height = img_width * aspect
                        c.drawImage(image_path, x_position, y_position - img_height/2,
                                  width=img_width, height=img_height, preserveAspectRatio=True)
                        x_position += img_width + 10
                    except Exception as e:
                        print(f"Error drawing image: {e}")
                    
                    last_pos = end
                
                # Draw remaining text
                suffix_text = text[last_pos:].strip()
                if suffix_text:
                    if x_position + stringWidth(suffix_text, default_font, 16) > width - left_margin:
                        y_position -= line_spacing
                        x_position = left_margin
                    c.drawString(x_position, y_position, suffix_text)
            
            else:
                # Draw regular text
                c.drawString(x_position, y_position, text)
            
            y_position -= line_spacing
            
            if y_position <= min_y_position:
                c.showPage()
                c.setFont(default_font, 16)
                y_position = top_margin
        
        c.showPage()
        c.setFont(default_font, 16)
    
    try:
        c.save()
        print(f"PDF saved successfully: {filename}")
        
        # Cleanup temporary image files
        for file in os.listdir(latex_image_dir):
            if file.endswith(".png"):
                try:
                    os.remove(os.path.join(latex_image_dir, file))
                except:
                    pass
                    
    except Exception as e:
        print(f"Error saving PDF: {e}")

# Main execution
if __name__ == "__main__":
    try:
        # Read CSV file
        df = pd.read_csv("QUE4.csv")
        print(f"Successfully read CSV with {len(df)} rows")
        
        # Create PDF
        create_pdf_with_math(df)
        
    except Exception as e:
        print(f"Error in main execution: {e}")

Successfully read CSV with 14 rows
PDF saved successfully: output_with_math.pdf
Error in callback <function _draw_all_if_interactive at 0x780056582980> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found